This document will take the transcripts from the Parsing Standup document, clean them up a bit and give some descriptive stats on the data. 

In [ ]:
import pandas as pd
from pandas import DataFrame
from collections import Counter
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sw = stopwords.words('english')
import numpy as np
import re


This cell reads in the dataframe from the first python script so we can manipulate the data further. 

In [ ]:
df = pd.read_csv("DF_TO_LOAD.csv")
df.head()


This cell takes the csv column from the dataframe and then breaks it down to tokens so we can get descriptive statistics on the transcripts. 

In [ ]:
# data = []
tot_tok = []
unq_tok = []
avg_len = []
lex_div = []
tten = []
words = []
transcripts = []
    
for item in df["CSV"]:
    with open (item, "r", encoding="UTF-8") as infile:
        corpus = infile.readlines()

    total_tokens = 0
    unique_tokens = 0
    token_len = 0
    avg_token_len = 0
    lex_diversity = 0

    corpora = " ".join(corpus)
    transcripts.append(corpora) #use this for lang detect, we will remove later
    includedTokens = [w for w in corpora.split()]
    fold = [w.lower() for w in includedTokens if w.isalpha()]
    fold  = [w for w in fold if w not in sw]
    
# Get the descriptive stats
    total_tokens = len(fold)

    unique_tokens = len(set(fold))

    token_len = [len(w) for w in fold]
    avg_token_len = np.mean(token_len)

    lex_diversity = len(set(fold))/len(fold)

    fdtxt = FreqDist(fold)
    fdtxt = fdtxt.most_common(10)

    strtext = "".join([str(elem)for elem in fdtxt])
    top_10 = strtext

    tot_tok.append(total_tokens)
    unq_tok.append(unique_tokens)
    avg_len.append(avg_token_len)
    lex_div.append(lex_diversity)
    tten.append(top_10)
    words.append(fold)

Now we take the lists that we created and add it to our dataframe so we can produce an output csv file.

In [ ]:
df["Total_Tokens"] = tot_tok
df["Unique_Tokens"] = unq_tok
df["Avg_Token_Length"] = avg_len
df["Lex_Diversity"] = lex_div
df["Most_Frequent_Words"] = tten
df["Words"] = words
df["Transcripts"] = transcripts



In [ ]:
from langdetect import detect

# Dectect language based on the first 500 characters and create a language column in the df
df['language'] = df.Transcripts.apply(lambda x: detect(x[:500]))

print(df.language.value_counts())
df.head()
#we want to get rid of any non-english specials as it doesn't help any analysis that we will be doing.
df = df.drop(df.index[df['language']!= "en"])    
del df["Transcripts"]

In [ ]:
df.head(4)

In [ ]:
df.to_csv("DataSetShare.csv", index = False)

Just messing around down here, can ignore, but may be interesting.


In [ ]:
from wordcloud import WordCloud

print(df.title[60])
wordcloud = WordCloud(background_color="Black", max_words=5000, contour_width=3, contour_color='midnightblue')
wordcloud.generate(' '.join(df.Words[60]))
wordcloud.to_image()
